In [ ]:
# Pronóstico usando un modelo autoregresivo
import warnings
warnings.filterwarnings("ignore")
# Carga de datos
import nbimporter  #  type: ignore
import functions  #  type: ignore
df_orig = functions.load_data()
df_orig.head()

In [ ]:
# Remove trend and cycle to make the series stationary
df_orig = functions.remove_trend_and_cycle(df_orig, yt_true_name="yt_true")
df_orig.head(20)

In [ ]:
# Construcción de la matriz de regresores
p_max = 13
df_orig = functions.make_lagged_ts(
    df=df_orig,
    p_max=p_max,
    y_column="yt_true_d1d12",
    fmt="lagged_{}m",
)
df_orig.head(20)

In [4]:
# Remoción de los valores faltantes
df_dropna = df_orig.dropna()

In [ ]:
# División de los datos en entrenamiento y prueba
(
    X_complete,
    y_complete,
    X_train,
    y_train,
    X_test,
    y_test,
) = functions.train_test_split(
    df=df_dropna,
    x_columns=[f"lagged_{i}m" for i in range(1, 14)],
    y_column="yt_true_d1d12",
)

X_complete.head()

In [ ]:
# Pronóstico usando regresión lineal
from sklearn.linear_model import LinearRegression
def create_pipeline():
    """Creates a linear regression model."""
    return LinearRegression()

# Entrenamiento y pronostico
hidden = 4
pipeline = create_pipeline()
pipeline.fit(X_train, y_train)
df_dropna[f"yt_d1d12_ar{p_max}"] = pipeline.predict(X_complete)
df_orig.loc[df_dropna.index, f"yt_d1d12_ar{p_max}"] = df_dropna[f"yt_d1d12_ar{p_max}"]
df_orig[["yt_true", "yt_true_d1d12", f"yt_d1d12_ar{p_max}"]].head(40)

In [ ]:
# Integración simple y estacional
import numpy as np  #  type: ignore
df_orig = df_orig.assign(yt_pred_d1d13_ar13=np.nan)
df_orig.loc[df_dropna.index, "yt_pred_d1d13_ar13"] = df_dropna["yt_d1d12_ar13"]
df_orig["yt_pred_d1d13_ar13"] += df_orig["yt_true"].shift(1)
df_orig["yt_pred_d1d13_ar13"] += df_orig["yt_true"].shift(12)
df_orig["yt_pred_d1d13_ar13"] -= df_orig["yt_true"].shift(13)
df_orig[["yt_true", "yt_pred_d1d13_ar13"]].head(40)

In [ ]:
# Gráfico de los pronósticos
functions.plot_time_series(df=df_orig, yt_col="yt_true")

In [ ]:
# Almacenamiento de los resultados
functions.save_forecasts(df_orig)
metrics = functions.compute_evaluation_metrics(df_orig.dropna())
functions.save_metrics(metrics)
metrics